## RagAssistant

#### Dependendecies

In [2]:
! pip install swarmauri[full]==0.1.88

   ---------------------------------------- 0.0/218.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/218.6 kB ? eta -:--:--
   ----- --------------------------------- 30.7/218.6 kB 445.2 kB/s eta 0:00:01
   ------------ -------------------------- 71.7/218.6 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 218.6/218.6 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: swarmauri
    Found existing installation: swarmauri 0.1.87
    Uninstalling swarmauri-0.1.87:
      Successfully uninstalled swarmauri-0.1.87


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
swarmauri-developer 0.0.8 requires swarmauri[full]==0.1.80, but you have swarmauri 0.1.88 which is incompatible.


In [1]:
import sqlite3
import gradio as gr
import pandas as pd
import json
from typing import List, Dict
from swarmauri.standard.models.concrete.OpenAIModel import OpenAIModel
from swarmauri.standard.models.concrete.GroqModel import GroqModel
from swarmauri.standard.models.concrete.MistralModel import MistralModel
from swarmauri.standard.vector_stores.concrete.TFIDFVectorStore import TFIDFVectorStore
from swarmauri.standard.vector_stores.concrete.Doc2VecVectorStore import Doc2VecVectorStore
from swarmauri.standard.vector_stores.concrete.MLMVectorStore import MLMVectorStore
from swarmauri.standard.conversations.concrete.LimitedSystemContextConversation import LimitedSystemContextConversation
from swarmauri.standard.agents.concrete.RagAgent import RagAgent
from typing import Any, Optional, Union, Dict
from swarmauri.core.messages import IMessage
from swarmauri.core.models.IModel import IModel
from swarmauri.standard.conversations.base.SystemContextBase import SystemContextBase
from swarmauri.standard.agents.base.VectorStoreAgentBase import VectorStoreAgentBase
from swarmauri.standard.vector_stores.base.VectorDocumentStoreRetrieveBase import VectorDocumentStoreRetrieveBase
from swarmauri.standard.documents.concrete.Document import Document
from swarmauri.standard.documents.concrete.EmbeddedDocument import EmbeddedDocument
from swarmauri.standard.messages.concrete import (HumanMessage, 
                                                  SystemMessage,
                                                  AgentMessage)
from swarmauri.standard.utils.load_documents_from_json import load_documents_from_json_file

C:\Users\bigman\anaconda3\envs\swarmauri\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


#### Project Class

In [2]:
class RagAssistant:
    def __init__(self, api_key: str = "", db_path='prompt_responses.db', 
                 vector_store_path=None, model_name = "openai_gpt-4-0125-preview"):
        print('Initializing... this will take a moment.')
        self.api_key = api_key
        self.db_path = db_path
        self.conversation = LimitedSystemContextConversation(max_size=36, system_message_content="")
        self.allowed_models = ['groq_llama3-8b-8192',
                            'groq_llama3-70b-8192',
                            'groq_mixtral-8x7b-32768',
                            'groq_gemma-7b-it',
                            'mistral_mistral-medium-latest',
                            'mistral_mistral-small-latest',
                            'mistral_open-mixtral-8x22b',
                            'mistral_open-mixtral-8x7b',
                            'mistral_open-mistral-7b',
                            'mistral_mistral-large-latest',
                            'openai_gpt-3.5-turbo',
                            'openai_gpt-3.5-turbo-16k',
                            'openai_gpt-4-0125-preview'
                              ]
            
        self.model = None
        self.retrieval_table = []
        self.document_table = []
        self.long_term_memory_df = None
        self.last_recall_df = None
        self.agent = self.initialize_agent()
        self.set_model(model_name)
        self.css = """
#chat-dialogue-container {
    min-height: 54vh !important;
}

#document-table-container {
    min-height: 80vh !important;
}

footer {
    display: none !important;
}
"""
        self.setup_gradio_interface()
        
    def initialize_agent(self):
        VS = Doc2VecVectorStore()
        agent = RagAgent(name="Rag", model=self.model, conversation=self.conversation, vector_store=VS)
        return agent

    def set_model(self, provider_model_choice: str):
        if provider_model_choice in self.allowed_models:
            provider, model_name = provider_model_choice.split('_')
            if provider == 'groq':
                self.model = GroqModel(api_key=self.api_key, model_name=model_name)
                
            if provider == 'mistral':
                self.model = MistralModel(api_key=self.api_key, model_name=model_name)
                
            if provider == 'openai':
                self.model = OpenAIModel(api_key=self.api_key, model_name=model_name)
                
            self.agent.model = self.model
                
        else:
            raise ValueError(f"Model name '{model_name}' is not supported. Choose from {self.allowed_models}")

    def change_vectorizer(self, vectorizer: str):
        if vectorizer == 'Doc2Vec':
            self.agent.vector_store = Doc2VecVectorStore()
        if vectorizer == 'MLM':
            self.agent.vector_store = MLMVectorStore()
        else:
            self.agent.vector_store = TFIDFVectorStore()
            
    
    def load_and_filter_json(self, file_info):
        # Load JSON file using json library
        try:
            documents = load_documents_from_json_file(file_info.name)
            self.agent.vector_store.documents = []
            self.agent.vector_store.add_documents(documents)
            self.long_term_memory_df = self.preprocess_documents(documents)
            return self.long_term_memory_df
        except json.JSONDecodeError:
            return "Invalid JSON file. Please check the file and try again."

    def preprocess_documents(self, documents):
        try:
            docs = [d.to_dict() for d in documents]
            for d in docs:
                metadata = d['metadata']
                for each in metadata:
                    d[each] = metadata[each]
                del d['metadata']
                del d['type']
                del d['embedding']
            df = pd.DataFrame.from_dict(docs)
            return df
        except Exception as e:
            print(f"postprocess_documents: {e}")

    
    def sql_log(self, prompt, response):
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute('''CREATE TABLE IF NOT EXISTS prompts_responses
                            (id INTEGER PRIMARY KEY AUTOINCREMENT, prompt TEXT, response TEXT)''')
            cursor.execute('''INSERT INTO prompts_responses (prompt, response) VALUES (?, ?)''', (prompt, response))
            conn.commit()
            conn.close()
        except:
            raise
    
    def save_df(self, df):
        documents = self.dataframe_to_documents(df)
        self.agent.vector_store.documents = []
        self.agent.vector_store.add_documents(documents)
        self.long_term_memory_df = self.preprocess_documents(documents)
        return self.long_term_memory_df

    def dataframe_to_documents(self, df):
        documents = []
        for index, row in df.iterrows():
            row_dict = row.to_dict() 
            id = row_dict.pop("id", "")
            content = row_dict.pop("content", "")
            metadata = row_dict  # remaining data becomes metadata
        
            # Convert the row to dictionary and create a DocumentBase instance
            
            document = EmbeddedDocument(id=id, content=content, metadata=metadata)
            documents.append(document)
        return documents
    
    async def chatbot_function(self, 
                         message, 
                         history, 
                         api_key: str = None, 
                         model_name: str = None, 
                         top_k: int = 5, 
                         temperature: int = 1, 
                         max_tokens: int = 256):
        try:
            if self.agent.vector_store.document_count() == 0:
                return "", [], [(message, "⚠️ Add Documents First")]
            else:

                
                
                # Set additional parameters
                self.api_key = api_key
                self.set_model(model_name)
                print(self.model, self.model.model_name, self.api_key)
                
                
                # Predict
                response = self.agent.exec(message, top_k=top_k, model_kwargs={'temperature': temperature, 'max_tokens': max_tokens})
                
                self.sql_log(message, response)
    
                # Update Retrieval Document Table
                self.last_recall_df = self.preprocess_documents(self.agent.last_similar_documents)
                
                # Get History
                history = [each['content'] for each in self.agent.conversation.to_dict()][1:]
                history = [(history[i], history[i+1]) for i in range(0, len(history), 2)]

                return "", self.last_recall_df, history
        except Exception as e:
            gr.Error(f"{e}")
            print(f"chatbot_function error: {e}")
            return "", [], history
    
    def clear_chat(self):
        self.agent.conversation.clear_history()
        return "", [], []
        
    def setup_gradio_interface(self):
        with gr.Blocks(css=self.css) as self.retrieval_table:
            with gr.Row():
                self.retrieval_table = gr.Dataframe(interactive=False, wrap=True, line_breaks=True, elem_id="document-table-container")

        
        with gr.Blocks(css=self.css) as self.chat:
            with gr.Row():
                self.chat_history = gr.Chatbot(label="Chat History", 
                                           layout="panel", 
                                           elem_id="chat-dialogue-container", 
                                           container=True, 
                                           show_copy_button=True,
                                           height="70vh")
            with gr.Row():
                self.input_box = gr.Textbox(label="Type here:", scale=6)
                self.send_button = gr.Button("Send", scale=1)
                self.clear_button = gr.Button("Clear", scale=1)
                
            with gr.Accordion("See Details", open=False):
                self.additional_inputs = [
                    gr.Textbox(label="API Key", value=self.api_key or "Enter your API Key"),
                    gr.Dropdown(self.allowed_models, 
                                value="openai_gpt-3.5-turbo", 
                                label="Model",
                                info="Select openai model"),
                    gr.Slider(label="Top K", value=10, minimum=0, maximum=100, step=5, interactive=True),
                    gr.Slider(label="Temperature", value=1, minimum=0.0, maximum=1.5, step=0.05, interactive=True),
                    gr.Slider(label="Max new tokens", value=256, minimum=256, maximum=4096, step=64, interactive=True)
                ]
    
    
            submit_inputs = [self.input_box, self.chat_history]
            submit_inputs.extend(self.additional_inputs)
            # Function to handle sending messages
            self.send_button.click(
                self.chatbot_function, 
                inputs=submit_inputs, 
                outputs=[self.input_box, self.retrieval_table, self.chat_history]
            )
        
            # Function to handle clearing the chat
            self.clear_button.click(
                self.clear_chat, 
                inputs=[], 
                outputs=[self.input_box, self.retrieval_table, self.chat_history]
            )

        
        with gr.Blocks(css=self.css) as self.document_table:
            with gr.Row():
                self.file = gr.File(label="Upload JSON File")
                self.vectorizer = gr.Dropdown(choices=["Doc2Vec", "TFIDF", "MLM"], value="Doc2Vec", label="Select vectorizer")
                self.load_button = gr.Button("load")
            with gr.Row():
                self.data_frame = gr.Dataframe(interactive=True, wrap=True, line_breaks=True, elem_id="document-table-container")
            with gr.Row():
                self.save_button = gr.Button("save")
                
            self.vectorizer.change(self.change_vectorizer, inputs=[self.vectorizer], outputs=self.data_frame)
            self.load_button.click(self.load_and_filter_json, inputs=[self.file], outputs=self.data_frame)
            self.save_button.click(self.save_df, inputs=[self.data_frame])


       
        self.app = gr.TabbedInterface([self.chat, self.retrieval_table, self.document_table], ["chat", "retrieval", "documents"])

    
    def launch(self):
        self.app.launch(share=False)

if __name__ == "__main__":
    rag_assistant = RagAssistant()
    rag_assistant.launch()

Initializing... this will take a moment.
Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


<swarmauri.standard.models.concrete.MistralModel.MistralModel object at 0x000001FFBE5C9720> open-mistral-7b YT3L87lZxul0ep0U5MjyCxsLwzLys8jr
